In [1]:
import spotipy
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from spotipy.oauth2 import SpotifyClientCredentials 
from sklearn.metrics.pairwise import cosine_similarity
from difflib import SequenceMatcher
client_id = "0054a24f2fc643c69d56d020dd5f70be"
client_secret = "98b4a4b772ad4eca934a92ca60c246a0"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [2]:
import mysql.connector

# Kết nối tới MySQL
def getConnect():
    mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="quoc0562138187",
    database="musicmanage")
    return mydb

In [40]:
#Truy vấn dữ liệu từ cơ sở dữ liệu
sql = """ 
  select s.id, s.title, s.lyrics,genres , group_concat(ar.name separator ', ') as artist, album from 
  (
  SELECT s.id, s.title, s.lyrics, 
  group_concat(gs.name separator ', ') as genres, al.name as album
  FROM songs as s
  inner join genre_of as go on s.id = go.song_id
  inner join genres as gs on go.genre_id = gs.id
  left join albums as al on s.album_id = al.id
  group by title ) as s
  inner join song_by as sb on s.id = sb.song_id
  inner join artists as ar on sb.artist_id = ar.id
  group by title
  """

# Đọc dữ liệu từ MySQL và chuyển đổi thành DataFrame
data = pd.read_sql(sql, getConnect())


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19088\536532418.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(sql, getConnect())


In [2]:
def similarity(a, b):
    a = a.lower()
    b = b.lower()
    return SequenceMatcher(None, a, b).ratio()

In [3]:
def search(title, artist):
    search = title + " " + artist
    results = sp.search(search,limit=50, offset=0 ,type="track",market="VN")
    items = results['tracks']['items']
    maxSimilarity = 0
    track_id =  0
    for i in range(20):
        similarTitle = similarity(title, items[i]['name'])
        similarArtist = similarity(artist, items[i]['artists'][0]['name'])
        similar = 2*similarTitle + similarArtist
        if (similar > maxSimilarity):
            maxSimilarity = similar
            track_id = items[i]['id']
    return track_id

In [4]:
def features_extract(title, artist):
    """
        Trích xuất đặc trưng của bài hát dựa vào API SPOTIFY
    """
    track_id = search(title, artist)
    features = sp.audio_features([track_id])[0]
    result = {}
    result['acousticness'] = features['acousticness']
    result['danceability'] = features['danceability'] 
    result['energy'] = features['energy']
    result['instrumentalness'] = features['instrumentalness']
    result['liveness'] = features['liveness']
    result['loudness'] = features['loudness']
    result['speechiness'] = features['speechiness']
    result['tempo'] = features['tempo']
    result['valence'] = features['valence']
    return result

In [5]:
features_extract("Bạc Phận", "Jack - J97")

{'acousticness': 0.539,
 'danceability': 0.801,
 'energy': 0.748,
 'instrumentalness': 7.66e-06,
 'liveness': 0.0674,
 'loudness': -7.128,
 'speechiness': 0.0323,
 'tempo': 119.972,
 'valence': 0.966}

In [42]:
features_df = data.apply(lambda row: features_extract(row['title'], row['artist']), axis= 1).apply(pd.Series)
features_df.head()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,0.708,0.408,0.193,0.528000,0.112,-19.728,0.0396,87.845,0.296
1,0.192,0.712,0.497,0.000002,0.198,-5.540,0.0295,81.878,0.563
2,0.739,0.798,0.791,0.000000,0.315,-8.197,0.0466,120.048,0.808
3,0.738,0.182,0.212,0.711000,0.104,-11.068,0.0321,95.882,0.160
4,0.574,0.675,0.642,0.068200,0.286,-8.496,0.0347,129.975,0.520


In [43]:
data = pd.concat([data, features_df], axis=1)
# Xuất dữ liệu ra file CSV
data.to_csv("song_database.csv", index=False)
data.head()

,id,title,lyrics,genres,artist,album,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,123,Ai muốn nghe không - dongvui harmony,"Đêm nay không ở nhà, không ở trên giường ngủ\r...","Nhạc rap, Nhạc trẻ",Đen,dongvui harmony,0.708,0.408,0.193,0.528000,0.112,-19.728,0.0396,87.845,0.296
1,199,Ai Thương Con Nhiều Hơn,,Nhạc thiếu nhi,Nguyệt Hằng,Nhong Nhong Nhong,0.192,0.712,0.497,0.000002,0.198,-5.540,0.0295,81.878,0.563
2,133,Anh Ba Hưng,Có anh Ba Hưng vốn thiệt nông dân\r\nĐi lính h...,"Nhạc cách mạng, Nhạc trữ tình, Nhạc quê hương",Cẩm Ly,Cô gái mở đường,0.739,0.798,0.791,0.000000,0.315,-8.197,0.0466,120.048,0.808
3,124,Anh Đếch Cần Gì Nhiều Ngoài Em - dongvui harmony,Và anh đếch cần gì nhiều ngoài em\r\nVà anh đế...,"Nhạc trẻ, Nhạc rap",Đen,dongvui harmony,0.738,0.182,0.212,0.711000,0.104,-11.068,0.0321,95.882,0.160
4,74,Anh Ghét Làm Bạn Em,Nhìn em đang thật vui sánh đôi bên người\r\nAn...,"Nhạc Pop, Nhạc trẻ",Phan Mạnh Quỳnh,Anh Ghét Làm Bạn Em,0.574,0.675,0.642,0.068200,0.286,-8.496,0.0347,129.975,0.520


In [3]:
df = pd.read_csv("song_database.csv")
df.head()

,id,title,lyrics,genres,artist,album,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,123,Ai muốn nghe không - dongvui harmony,"Đêm nay không ở nhà, không ở trên giường ngủ\r...","Nhạc rap, Nhạc trẻ",Đen,dongvui harmony,0.708,0.408,0.193,0.528000,0.112,-19.728,0.0396,87.845,0.296
1,199,Ai Thương Con Nhiều Hơn,NaN,Nhạc thiếu nhi,Nguyệt Hằng,Nhong Nhong Nhong,0.192,0.712,0.497,0.000002,0.198,-5.540,0.0295,81.878,0.563
2,133,Anh Ba Hưng,Có anh Ba Hưng vốn thiệt nông dân\r\nĐi lính h...,"Nhạc cách mạng, Nhạc trữ tình, Nhạc quê hương",Cẩm Ly,Cô gái mở đường,0.739,0.798,0.791,0.000000,0.315,-8.197,0.0466,120.048,0.808
3,124,Anh Đếch Cần Gì Nhiều Ngoài Em - dongvui harmony,Và anh đếch cần gì nhiều ngoài em\r\nVà anh đế...,"Nhạc trẻ, Nhạc rap",Đen,dongvui harmony,0.738,0.182,0.212,0.711000,0.104,-11.068,0.0321,95.882,0.160
4,74,Anh Ghét Làm Bạn Em,Nhìn em đang thật vui sánh đôi bên người\r\nAn...,"Nhạc Pop, Nhạc trẻ",Phan Mạnh Quỳnh,Anh Ghét Làm Bạn Em,0.574,0.675,0.642,0.068200,0.286,-8.496,0.0347,129.975,0.520


In [14]:
ids = [197,159,132,74,165,118,178,35,13,145]

In [17]:
df[['id','title']].loc[df['id'].isin(ids)]

,id,title
4,74,Anh Ghét Làm Bạn Em
26,132,Bên Trên Tầng Lầu
27,165,Biển Cạn
31,178,Cắc Tùng Cắc Tùng
57,118,Đêm Nằm Mơ Phố
88,13,Lạc Trôi( Masew Mix)
104,159,nàng thơ
108,35,Ngoài 30
120,145,Qua Sông
134,197,Tiễn Biệt


In [7]:
# One hot encoding 
def oneHotEncoding(df, column):
    col_ohe = df[column].str.get_dummies(', ')
    return col_ohe

In [8]:
def create_feature_data(df):
    """ 
        Xử lý dữ liệu để tạo tập dữ liệu đặc trưng cuối cùng sử dụng cho khuyến nghị
    """
    # One hot encoding cột thể loại
    genre_OHE = oneHotEncoding(df,'genres')
    # Lấy các cột cần chuẩn hoá
    float_cols = df.dtypes[df.dtypes == 'float64'].index.values
    
    # Chuẩn hoá các cột đăng trưng audio
    scaler = MinMaxScaler()
    floats = df[float_cols].reset_index(drop = True)
    df_scaled = pd.DataFrame(scaler.fit_transform(floats), columns=floats.columns, index=floats.index)
    final = pd.concat([df_scaled,genre_OHE],axis=1)
    final['id']=df['id'].values
    return final
    

In [47]:
final_features_data = create_feature_data(df)
final_features_data.to_csv("final_features_data.csv", index = False)
final_features_data.head()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,Nhạc Acoustic,Nhạc Pop,Nhạc Remix,Nhạc cách mạng,Nhạc quê hương,Nhạc rap,Nhạc thiếu nhi,Nhạc trẻ,Nhạc trữ tình,id
0,0.724684,0.302949,0.104260,0.742616,0.088248,0.264268,0.093349,0.204807,0.248471,0,0,0,0,0,1,0,1,0,123
1,0.180380,0.710456,0.445067,0.000003,0.178168,0.826569,0.034422,0.165159,0.545303,0,0,0,0,0,0,1,0,0,199
2,0.757384,0.825737,0.774664,0.000000,0.300502,0.721267,0.134189,0.418776,0.817676,0,0,0,1,1,0,0,0,1,133
3,0.756329,0.000000,0.125561,1.000000,0.079883,0.607483,0.049592,0.258207,0.097276,0,0,0,0,0,1,0,1,0,124
4,0.583333,0.660858,0.607623,0.095921,0.270180,0.709417,0.064761,0.484735,0.497499,0,1,0,0,0,0,0,1,0,74


In [4]:
final_features_data= pd.read_csv("final_features_data.csv")

In [21]:
def playlist(playlistId):
    mydb = getConnect()
    sqlPlaylist = f"SELECT song_id as id, date_added FROM playlist_includes WHERE playlist_id = {playlistId}"
    return pd.read_sql(sqlPlaylist,mydb)

In [5]:
# Lấy lịch sử nghe nhạc của người dùng
def history(userId):
    mydb = getConnect()
    sqlHistory = f""" 
        SELECT songs.id, songs.title, max(date) as date FROM history_listens 
        INNER JOIN songs ON songs.id = history_listens.song_id
        where user_id = {userId}
        group by song_id 
        order by date desc 
        limit 10
        """
    return pd.read_sql(sqlHistory, mydb)

In [17]:
playlistId = 20
playlist_df = playlist(playlistId)
playlist_df.rename(columns={'date_added': 'date'}, inplace=True)
playlist_df

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_19328\3095150309.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sqlPlaylist,mydb)


,id,date
0,27,2023-04-26 22:21:44
1,35,2023-04-26 22:21:44
2,37,2023-04-26 22:21:44
3,38,2023-04-26 22:21:44
4,39,2023-04-26 22:21:44
5,40,2023-04-26 22:21:44
6,42,2023-04-26 22:21:44
7,44,2023-04-26 22:21:44
8,45,2023-04-26 22:21:44
9,52,2023-04-26 22:21:44


In [6]:
userId = 6
history_listen_df = history(userId)
history_listen_df

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10632\853795152.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sqlHistory, mydb)


,id,title,date
0,159,nàng thơ,2023-04-26 22:09:32
1,160,từ thích thích thành thương thương,2023-04-26 22:09:30
2,88,yêu thì yêu không yêu thì yêu,2023-04-26 22:09:28
3,87,anh nhà ở đâu thế,2023-04-26 22:09:26
4,35,Ngoài 30,2023-04-10 00:19:59
5,44,Chỉ Một Đêm Nữa Thôi,2023-04-09 22:09:24
6,38,Bạc Phận,2023-04-09 21:13:06
7,37,Đắp Mộ Cuộc Tình,2023-04-09 19:18:03
8,12,Chạy Ngay Đi (Sky Tour 2019),2023-04-09 19:14:32
9,27,They Said,2023-04-09 18:55:19


In [7]:
def generate_history_feature(final_features_data, history_listen_df ):
    """
        Tóm tắt lịch sử nghe nhạc của người dùng thành một vector
    """
    # tìm đặc trưng của mỗi bài hát trong lịch sử
    final_features_history = final_features_data[final_features_data['id'].isin(history_listen_df['id'].values)]
    # tìm tất cả bài hát không có trong lịch sử
    final_features_data_non_history = final_features_data[~final_features_data['id'].isin(history_listen_df['id'].values)]
    result = final_features_history.drop(columns = "id")
    return result.sum(axis = 0), final_features_data_non_history


In [24]:
# tạo các đặc trưng
playlist_df_vector, final_features_data_non_playlist = generate_history_feature(final_features_data, playlist_df)

In [8]:
# tạo các đặc trưng
history_listen_vector, final_features_data_non_history = generate_history_feature(final_features_data, history_listen_df)

In [9]:
history_listen_vector

acousticness        5.553797
danceability        6.365952
energy              5.195067
instrumentalness    0.000317
liveness            1.214659
loudness            7.546489
speechiness         3.483664
tempo               5.762350
valence             4.544747
Nhạc Acoustic       4.000000
Nhạc Pop            5.000000
Nhạc Remix          1.000000
Nhạc cách mạng      0.000000
Nhạc quê hương      0.000000
Nhạc rap            3.000000
Nhạc thiếu nhi      0.000000
Nhạc trẻ            9.000000
Nhạc trữ tình       1.000000
dtype: float64

In [ ]:
playlist_df_vector

acousticness         5.185443
danceability         7.225201
energy               7.615471
instrumentalness     0.257973
liveness             3.413634
loudness             9.073637
speechiness          2.811552
tempo                7.005521
valence              6.633685
Nhạc Acoustic        0.000000
Nhạc Pop             8.000000
Nhạc Remix           2.000000
Nhạc cách mạng       0.000000
Nhạc quê hương       0.000000
Nhạc rap             3.000000
Nhạc thiếu nhi       0.000000
Nhạc trẻ            11.000000
Nhạc trữ tình        1.000000
dtype: float64

In [ ]:
recommend = generate_recommended_song_list(df, playlist_df_vector, final_features_data_non_playlist)
recommend[['id','title']]

In [11]:
recommend = generate_recommended_song_list(df, history_listen_vector, final_features_data_non_history)
recommend[['id','title']]

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10632\3361857212.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_history_df['similarity'] = cosine_similarity(non_history_features.drop('id', axis = 1).values, features.values.reshape(1, -1))[:,0]


,id,title
143,36,Ưng Quá Chừng
111,78,Nhạt
28,76,Bóng Tối Không Em
4,74,Anh Ghét Làm Bạn Em
29,47,Bùa Yêu
62,50,Drama Queen
35,48,Chị Ngả Em Nâng
115,75,Nơi Ấy Con Tìm Về
93,175,LOVE SCENARIO
128,34,Thị Mầu


In [12]:
from sklearn.neighbors import NearestNeighbors

In [31]:
def NearestNeighbor(df, features, non_history_features):
    """
    Đề xuất các bài hát
    """
    nn = NearestNeighbors(n_neighbors=10)
    nn.fit(non_history_features.drop('id',axis = 1))
    neighbors = nn.kneighbors(features.values.reshape(1, -1),return_distance=False )[0]
    return df[['id','title']].iloc[neighbors]

In [32]:
NearestNeighbor(df,history_listen_vector, final_features_data_non_history)

c:\Users\ADMIN\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,id,title
116,52,Nói Thương Nhau Đừng Làm Trái Tim Em Đau
24,63,Bến Chiều
134,197,Tiễn Biệt
74,65,Gửi Em Chiếc Nón Bài Thơ
83,171,Hoa Sữa
88,13,Lạc Trôi( Masew Mix)
26,132,Bên Trên Tầng Lầu
109,193,Người Chồng Xa Xứ
48,180,Con Heo Đất
40,206,Chú Chó Bingo


In [ ]:
recommend_json = recommend['id'].to_json(orient='values')
json_dict = {"results": recommend_json}
json_dict

{'results': '[73,67,66,71,72,81,83,80,86,85]'}

In [ ]:
# Lấy thông tin bài hát vừa thêm
def getSongById(songId):
    sql = f""" 
    select s.id, s.title, s.lyrics,genres , group_concat(ar.name separator ', ') as artist, album from 
    (
    SELECT s.id, s.title, s.lyrics, 
    group_concat(gs.name separator ', ') as genres, al.name as album
    FROM songs as s
    inner join genre_of as go on s.id = go.song_id
    inner join genres as gs on go.genre_id = gs.id
    left join albums as al on s.album_id = al.id
    group by title ) as s
    inner join song_by as sb on s.id = sb.song_id
    inner join artists as ar on sb.artist_id = ar.id
    where s.id = {songId}
    group by title
    """
    return pd.read_sql(sql,getConnect())

In [ ]:
song_id = 74
song  = getSongById(song_id)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_11320\952135399.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql,getConnect())


In [ ]:
def getFeaturesOneSong(song):
    features_one_song = {}
    while len(features_one_song) == 0:
        try:
            features_one_song_df = features_extract(song['title'].values[0], song['artist'].values[0])
        except:
            print("Lỗi trích xuất đặc trưng")
    return features_one_song

In [ ]:
features_one_song = getFeaturesOneSong(song)
features_one_song_df = pd.DataFrame.from_dict(features_one_song,orient='index').T

Lỗi trích xuất đặc trưng


In [ ]:
song_features = pd.concat([song,features_one_song_df], axis=1)
song_features

,id,title,lyrics,genres,artist,album,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,74,Anh Ghét Làm Bạn Em,Nhìn em đang thật vui sánh đôi bên người\r\nAn...,Nhạc trẻ,Phan Mạnh Quỳnh,Anh Ghét Làm Bạn Em,0.574,0.675,0.642,0.0682,0.286,-8.496,0.0347,129.975,0.52


In [ ]:
genres = ['Nhạc trẻ, Nhạc trữ tình, Nhạc cách mạng, Nhạc thiếu nhi, Nhạc quê hương, Nhạc rap, Nhạc Remix, Nhạc Acoustic, Nhạc Pop']
genres_df = pd.DataFrame({'id' : 0, 'genres':genres})
song_features = pd.concat([song_features,genres_df]).reset_index(drop=True)

In [ ]:
def create_feature_data_one_song(song_features):
    df = pd.read_csv("song_database.csv")
    float_cols = df.dtypes[df.dtypes == 'float64'].index.values
    genre_OHE = oneHotEncoding(song_features,'genres')
    floats = df[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    scaler.fit(floats)
    song_features_scaled  = pd.DataFrame(scaler.transform(song_features[float_cols] ), columns=floats.columns)
    final = pd.concat([song_features_scaled,genre_OHE],axis=1)
    final['id']=song_features['id'].values
    return final.iloc[[0]]

In [ ]:
complete_song_features = create_feature_data_one_song(song_features)

In [ ]:
complete_song_features.to_csv('final_features_data.csv', mode='a', header=False, index=False)